# Runtime Inference Acceleration 数学基础

- 作者：ChatGPT + Vesper
- 版本：v0.1
- 用途： Runtime Inference Acceleration 的数学基础，补齐数学知识


# 第 0 章：整体视图 —— Runtime Inference 的 9 大数学支柱

在现代推理系统中（LLM、Transformer、CNN、语音模型等），涉及到的数学大致可以归纳为 9 大类：

1. **线性代数（Linear Algebra）**  
   - 矩阵乘法（GEMM）、SVD、低秩分解、卷积表达
2. **数值优化（Numerical Optimization）**  
   - 剪枝、量化、蒸馏背后的优化目标与约束
3. **近似理论（Approximation Theory）**  
   - 量化噪声建模、激活函数近似、cheap ops 替代
4. **概率与统计（Probability & Statistics）**  
   - KL、温度缩放、分布拟合、分位数剪裁
5. **信息论（Information Theory）**  
   - 熵、编码、压缩极限、rate–distortion 视角
6. **信号处理与卷积数学（Signal Processing）**  
   - FFT、Winograd、Toeplitz/Circulant 结构
7. **计算图与图论（Computational Graph & Graph Theory）**  
   - DAG、图重写、kernel fusion、memory planning
8. **数值稳定性与复杂度（Numerical Stability & Complexity）**  
   - 浮点误差、softmax 稳定写法、复杂度分析
9. **硬件相关数学（Hardware-Aware Math）**  
   - Roofline 模型、tiling、SIMD/FMA、GEMM 内核
        |

这些模块之间的关系可以大致用一张“心智地图”表示：

```text
              ┌─────────┐
              │ 线性代数 │  ← 关键：GEMM / SVD / Conv=MatMul
              └────┬────┘
                   │
          ┌────────┴─────────┐
          ▼                  ▼
   数值优化(剪枝/量化)   近似理论(激活/量化误差)
          │                  │
          ├────────┬─────────┤
          ▼        ▼         ▼
       概率统计  信息论   信号处理(Conv/FFT)
          │                  │
          └────────┬─────────┘
                   ▼
          计算图 & 图论 (fusion, scheduling)
                   │
                   ▼
     数值稳定性 & 硬件数学 (FP16/FP8, Roofline, tiling)
```

> 本 notebook 后续章节将按照这个结构展开。  
> 第 1 章从 **线性代数** 开始，因为这是推理加速中最核心、出现频率最高的数学语言。


# 第 1 章：线性代数（Linear Algebra for Runtime Inference）

## 1.1 线性代数在推理加速中的角色

在深度学习推理中，绝大部分“重计算”都可以抽象为：

- 向量内积（dot product）
- 矩阵–向量乘法（GEMV）
- 矩阵–矩阵乘法（GEMM）
- 卷积经变换后变成的 GEMM（im2col / Winograd）

例如：

- 全连接层（Fully Connected）：  
  $y = W x + b$
- Transformer MLP：  
  $Y = \sigma(X W_1 + b_1) W_2 + b_2$
- 自注意力（Self-Attention）：  
  $Q = X W_Q, \quad K = X W_K, \quad V = X W_V$  
  $A = \text{softmax}(QK^\top / \sqrt{d_k})$  
  $O = A V$

**【工程视角】**  
推理加速库（cuBLAS、CUTLASS、MKL、QNNPACK、TensorRT 等）的核心任务，就是：

> 让这些矩阵/向量运算在特定硬件上尽可能高效地执行。

理解线性代数 → 理解：

- 为什么所有人都在说 GEMM
- 为什么 low-rank / SVD / LoRA 能够“白嫖”加速
- 为什么 Conv2d 经常被变成 GEMM
- 为什么 Attention 的优化集中在 QKᵀ / AV 上

---

## 1.2 从内积到 GEMM：推理的基本算子

### 1.2.1 内积（dot product）

给定两个向量 $a,b \in \mathbb{R}^d$，它们的内积为：

$$
\langle a, b \rangle = \sum_{i=1}^d a_i b_i
$$


这是最小的“乘–加”单元。

**【硬件视角】**  
现代 CPU/GPU/NPU 都有 **FMA (Fused Multiply-Add)** 指令：

$$
\text{FMA}(a,b,c) = a \times b + c
$$


dot product 可以写成：

```text
acc = 0
for i in range(d):
    acc = fma(a[i], b[i], acc)
```

- FMA 是矩阵乘法的底层原语（primitive）
- 越能让硬件长时间“刷 FMA”，推理吞吐越高
- 数学上的意义：
$\text{round}\big(\text{round}(a\times b) + c \big)
\neq
\text{round}(a\times b + c)$
FMA 使用右边的（更精确）。

---

### 1.2.2 矩阵乘法（GEMM）

GEMM 的一般形式：

$$
C = \alpha A B + \beta C
$$


其中：

- $A \in \mathbb{R}^{M \times K}$
- $B \in \mathbb{R}^{K \times N}$
- $C \in \mathbb{R}^{M \times N}$
- $\alpha, \beta$ 为标量（常见情况：$\alpha=1, \beta=0$）

展开单个元素：

$$
c_{ij} = \alpha \sum_{k=1}^K a_{ik} b_{kj} + \beta c_{ij}^{\text{(old)}}
$$


这就是：**多次 FMA 的二维版本**。

**【工程视角】**  
- 全连接层 = 多个 GEMM
- Attention = 2 个大 GEMM（QKᵀ 和 AV）+ 若干小算子
- Conv2d 经过变换后 = GEMM
- 推理加速的大头，就是把不同算子统一映射到高效 GEMM 内核上


小结
| 网络组件            | 数学形式     | 最终变成          |
| --------------- | -------- | ------------- |
| 全连接层 FC         | Wx + b   | GEMM          |
| 卷积 conv2d       | K * X    | im2col → GEMM |
| Attention QKᵀ   | QKᵀ      | GEMM          |
| Attention AV    | AV       | GEMM          |
| Transformer MLP | XW₁, XW₂ | GEMM          |
| Embedding       | 查表       | 选行（类似 GEMM）   

附： 为什么说“Embedding 类似 GEMM”？

虽然 数学上 Embedding 是选行，
但它可以看成一个非常稀疏的矩阵乘法。
给一个 embedding matrix（词向量矩阵）
$E \in \mathbb{R}^{V \times D}$
- V = vocabulary size（如 50k）

- D = hidden dimension（如 1024）

每一行 $𝐸_i$ 就是一个 token 的向量。当一个 token 的 ID = i 时，embedding 做的事情是：$\text{Embedding}(i) = E_{i}$
没有乘法，没有矩阵运算，所以 Embedding 的 FLOPs ~ 0（几乎 0）。
真正的成本是：

- 内存访问（memory bandwidth）

- 随机访问（random access）

- cache miss（embedding layer 很容易 miss cache）

假设 token id = 3， 这时可以构造一个 size V 的 one-hot 向量：$x = [0,0,0,1,0,0,\dots]^T \in \mathbb{R}^V$

Embedding 的输出就是：$y = x^T E \in \mathbb{R}^D$

展开来看：$y = \sum_{i=1}^V x_i E_i$

但因为只有一个 $X_i = 1$，其余全是 0： $y = E_3$

这本质上是一种特殊的 GEMM：$Y = X E$
其中 X 是 batch 个 one-hot 向量。

只是：

X 中绝大部分值是 0， X 只有一个 1， 所以矩阵乘法退化成“选行”.

这么理解的好处：
Embedding 从数学上可以视为 GEMM 的特例（稀疏 GEMM）

这让我们可以：

- 使用矩阵观点分析它

- 使用 GEMM 的 layout、tiling 理解其优化

- 把 embedding 也当成一个“矩阵算子”，可以融合、量化、cache友好化

总结一下，Embedding 的瓶颈不是算力（不是 FMA），而是：

- 内存带宽（bandwidth）

- 随机访问（random access patterns）

- cache line miss

因此：

常见看到优化策略是：

- 把 embedding matrix rearrange（行压缩）

- 把常用 token 提前放在 cache-friendly 区域

- 更小 embedding（量化、减少维度）

- batching lookups


---

### 1.2.3 张量运算：线性代数的多维版本

张量（tensor）本质就是多维数组，常见是 rank-3 / rank-4：

- rank-3：$B \times T \times D$（batch × seq_len × hidden_dim）
- rank-4：$B \times C \times H \times W$（用于卷积）

张量运算（einsum、batched matmul）最终都会被分解成：

- 一堆 reshape / transpose
- 若干矩阵乘法（batched GEMM）
- 再 reshape 回想要的形状

> 所以“学线性代数”不仅是学矩阵，还要学会**如何把高维张量 reshape 成矩阵**以便使用高效 GEMM。


## 1.3 低秩近似（Low-Rank Approximation）与 SVD

### 1.3.1 问题背景

给定一个大的权重矩阵 $W \in \mathbb{R}^{m \times n}$，例如：

- Transformer MLP 的权重（如 4096 × 11008）
- Attention 中的投影矩阵（如 4096 × 4096）

在很多实际模型中，$W$ 的“有效秩（effective rank）”远小于 $\min(m,n)$：
- 也就是说，“信息”集中在少数几个方向上。

这时我们希望用一个秩为 $k \ll \min(m,n)$ 的矩阵 $W_k$ 来近似 $W$：

$$
\min_{\text{rank}(X) \le k} \ \|W - X\|_F
$$


这个问题的解析解由 **奇异值分解（SVD）** 给出。

---

### 1.3.2 奇异值分解（SVD）的形式

$$
W = U \Sigma V^\top
$$


- $U \in \mathbb{R}^{m \times m}$：列向量为左奇异向量
- $V \in \mathbb{R}^{n \times n}$：列向量为右奇异向量
- $\Sigma \in \mathbb{R}^{m \times n}$：对角线为奇异值 $\sigma_1 \ge \sigma_2 \ge \dots \ge \sigma_r > 0$，其余为 0

截断到前 $k$ 个奇异值：

$$
W_k = U_k \Sigma_k V_k^\top
$$


- $U_k \in \mathbb{R}^{m \times k}$
- $\Sigma_k \in \mathbb{R}^{k \times k}$
- $V_k \in \mathbb{R}^{n \times k}$

这是一个秩为 $k$ 的矩阵。

---

### 1.3.3 最优低秩近似定理（Eckart–Young–Mirsky）

**定理：**  
在 Frobenius 范数下，秩不超过 $k$ 的矩阵中离 $W$ 最近的是 $W_k$：

$$
W_k = \arg\min_{\text{rank}(X) \le k} \|W - X\|_F
$$


而误差大小刚好等于被截掉的奇异值的平方和：

$$
\|W - W_k\|_F^2 = \sum_{i=k+1}^r \sigma_i^2
$$


**【直观解释】**  
奇异值越大，对矩阵“能量”的贡献越大；截断后，丢掉的能量恰好是对应奇异值平方的总和。

---

### 1.3.4 用两个小矩阵实现低秩近似

在工程中，为了减少 FLOPs，通常把 $W_k$ 拆成两个更小的矩阵相乘：

$$
W_k = A B, \quad A \in \mathbb{R}^{m \times k}, \ B \in \mathbb{R}^{k \times n}
$$


一种常见构造是：

$$
A = U_k \Sigma_k^{1/2}, \quad B = \Sigma_k^{1/2} V_k^\top
$$


于是：

$$
W x \approx W_k x = A (B x)
$$


- 原始计算：$W x$ 是一个 $m \times n$ 的矩阵–向量乘法，复杂度 $O(mn)$
- 分解后：
  - 先算 $z = B x$（维度 $k$）
  - 再算 $A z$
  - 总复杂度 $O(kn + mk)$

如果 $k \ll \min(m,n)$，总体 FLOPs 大幅减少。

**【工程视角】**  
- 这就是很多 **SVD-based compression / low-rank factorization / LoRA** 内核的数学基础。
- 在模型部署时，你会看到某些层被替换成两层小的 Linear：`Linear(d_in → k)` + `Linear(k → d_out)`。

---

### 1.3.5 文字图示（矩阵结构的可视化）

原始矩阵 $W$：

```text
W (m×n)
+------------------------+
| █ █ █ █ █ █ █ █ █ █ █  |
| █ █ █ █ █ █ █ █ █ █ █  |
| █ █ █ █ █ █ █ █ █ █ █  |
| █ █ █ █ █ █ █ █ █ █ █  |
|         ···            |
+------------------------+
```

低秩分解后：

```text
A (m×k)          B (k×n)
+------+       +------------------+
| █ █ |       | █ █ █ █ █ █ █ █   |
| █ █ |   x   | █ █ █ █ █ █ █ █   |
| █ █ |       |        ···        |
| █ █ |       +------------------+
+------+
```

推理时：

1. 先计算 $z = B x$（维度 k）  
2. 再计算 $y = A z$

当 k 比 m,n 小很多时，这是显著的加速。

---

### 1.3.6 在推理加速中的典型用法

1. **MLP 权重低秩分解**  
   - 原始：4096×11008 的巨矩阵  
   - 分解：4096×k 和 k×11008，k=1024 或更小  
2. **LoRA（Low-Rank Adaptation）**  
   - 把更新量 $\Delta W$ 参数化为 $A B$，rank 通常很小（如 8、16）  
   - 对部署来说，推理时多了两层小矩阵乘法
3. **KV Cache 压缩 / 投影**  
   - 用低秩映射降低 KV 维度，从而减少内存和带宽
4. **结构化剪枝的替代方案**  
   - 相比直接硬剪通道，低秩分解是更平滑的维度削减方式

> 在这类做边缘推理的场景中：  
> - 你会希望在 **不大幅降精度** 的前提下，**显著减少 MLP 和投影层的 FLOPs 和参数量**，低秩分解是最主流的数学工具之一。


## 1.4 矩阵范数与误差度量

在做任何压缩（低秩、剪枝、量化）时，你都需要一个“度量标准”来衡量：

- 压缩前后的权重差异有多大？
- 这会带来多大的输出误差？

矩阵范数提供了这些度量工具。

---

### 1.4.1 Frobenius 范数（整体能量）

$$
\|W\|_F = \sqrt{\sum_{i,j} w_{ij}^2}
$$


对应于把矩阵当成一个长向量后的 $L_2$ 范数。

- 它衡量整体“能量”（energy）
- 在 SVD 中有非常漂亮的性质：

$$
\|W\|_F^2 = \sum_{i=1}^r \sigma_i^2
$$


而对低秩近似：

$$
\|W - W_k\|_F^2 = \sum_{i=k+1}^r \sigma_i^2
$$


**【工程应用】**  
- 可以通过奇异值快速估计压缩误差的上界
- 可以比较不同 $k$ 值下的误差，做 bit–accuracy tradeoff

---

### 1.4.2 谱范数（operator norm / 2-范数）

$$
\|W\|_2 = \sigma_{\max}(W)
$$


- 等于最大的奇异值
- 反映：$W$ 作为线性算子能把向量放大的最大比例

具体来说：

$$
\|W\|_2 = \max_{\|x\|_2 = 1} \|W x\|_2
$$


**【工程意义】**  
- 如果 $\|W\|_2$ 非常大，则对输入的小扰动非常敏感  
**解释**：当矩阵的谱范数（spectral norm），即 $|W|_2$ 非常大时，它意味着这个矩阵作为一个线性变换，能够将输入向量“放大”的最大比例非常大。

谱范数的定义是：

$$
\|W\|_2 = \max_{\|x\|_2 = 1} \|W x\|_2
$$

这意味着，对于任何非零向量 $x$，

$$|W x|_2 \le |W|_2 |x|_2$$

现在，考虑一个输入向量 $x_0$ 受到一个小扰动 $\Delta x$。那么，输出的改变将是 $W(x_0 + \Delta x) - W x_0 = W \Delta x$。

输出扰动的范数是 $|W \Delta x|_2$。根据上面的不等式，我们可以得到：

$$|W \Delta x|_2 \le |W|_2 |\Delta x|_2$$

如果 $|W|_2$ 非常大，即使输入的扰动 $|\Delta x|_2$ 非常小，经过矩阵 $W$ 变换后，输出的扰动 $|W \Delta x|_2$ 也会被放大很多倍。这意味着原始输入中微小的误差或噪声，在经过这个矩阵层之后，可能会被显著放大，导致输出结果的剧烈变化。

总结来说：

大 $|W|_2$ 意味着强放大效应：矩阵 $W$ 有能力将某些方向上的输入向量放大很多倍。
误差传播：当输入存在小扰动时（比如量化误差、浮点数误差或传感器噪声），这个扰动会被 $|W|_2$ 的大小所放大，从而导致最终输出的误差也很大。
在推理系统中，尤其是在量化或低精度计算时，如果模型的某个权重矩阵的谱范数过大，那么该层将对量化噪声、数值舍入误差非常敏感，容易导致精度大幅下降或输出不稳定。因此，在模型设计或优化过程中，有时会通过正则化等方式来限制权重的谱范数，以提高模型的鲁棒性和数值稳定性。
- 在量化/剪枝时，这种层更容易出现数值不稳定 / 输出来回抖动  
- 有些鲁棒性分析会用谱范数做 Lipschitz 常数的上界

**解释**：

*什么是 Lipschitz 常数*： 在数学中，一个函数 $f: X \to Y$ 是 Lipschitz 连续的，如果存在一个常数 $K \ge 0$，使得对于 $X$ 域中的任意两个点 $x_1$ 和 $x_2$，都有：

$$\|f(x_1) - f(x_2)\| \le K \|x_1 - x_2\|$$
这个常数 $K$ 就被称为该函数的 Lipschitz 常数。这里的 $\|\cdot\|$ 表示某种范数（例如向量的欧几里得范数或矩阵的谱范数）。

*直观理解*： Lipschitz 常数 $K$ 给出了函数 “最陡峭” 的程度。它限制了函数值变化的速率：

如果 $K$很小，函数是“平缓”的，输出的变化不会比输入的微小变化大太多。它像一个斜率有限的函数。
如果 $K$很大，函数可能是“陡峭”的，输出的变化可能比输入的微小变化大很多。这意味着函数对输入的小扰动非常敏感。

*数值稳定性与误差传播*：如果一个矩阵（可以看作一个线性函数）的谱范数很大，那么它会将输入的微小误差放大很多倍。一个线性函数 $f(x) = Wx$ 的 Lipschitz 常数就是它的谱范数 $|W|_2$。
对于非线性函数（如激活函数、整个神经网络），其 Lipschitz 常数限制了输入噪声或量化误差在网络中传播时的最大放大倍数。
低 Lipschitz 常数 的模型通常具有更好的 数值稳定性和对抗样本的鲁棒性。

*模型压缩与量化*：在量化或剪枝过程中，我们会对模型参数引入微小的扰动（量化误差）。如果某一层或整个模型的 Lipschitz 常数过大，这些微小扰动就可能被显著放大，导致模型精度急剧下降。
因此，在进行模型压缩时，有时会关注保持或限制模型的 Lipschitz 常数，以确保压缩后的模型仍然稳定且性能良好。在实践中，直接计算复杂深度学习模型的精确 Lipschitz 常数通常非常困难（NP-hard）。
但可以通过一些方法来 估计其上界 或 正则化 来间接限制它，例如：谱范数正则化 (Spectral Normalization)：限制神经网络中权重矩阵的谱范数，从而限制线性层的 Lipschitz 常数。 某些激活函数（如 ReLU）是 1-Lipschitz 的，有助于控制传播的放大。Batch Normalization 或 Layer Normalization 也有助于稳定激活的分布，间接控制函数的行为。总而言之，Lipschitz 常数是衡量函数平滑度和对输入扰动敏感程度的一个重要指标。


---

### 1.4.3 $L_1$ 和 $L_0$ 范数（稀疏性）

- 元素级 $L_1$：

$$
\|W\|_1 = \sum_{i,j} |w_{ij}|
$$


- “$L_0$”（非零个数）：

$$
\|W\|_0 = \#\{(i,j) : w_{ij} \ne 0\}
$$


**【工程视角】**  

- $L_0$ 直接度量“有多少元素被保留/剪掉”，但对应的优化问题是 NP-hard
- 通常用 $L_1$ 作为凸近似：

$$
L(W) + \lambda \|W\|_1
$$


  训练后很多元素自然趋近于 0，方便做后处理剪枝。

**【具体解释】**
1. $L_0$ 范数：理想但难优化
定义：$\|W\|_0$ 严格来说不是一个范数，它表示矩阵 $W$ 中 非零元素的个数。
目标：在剪枝中，我们希望最小化非零元素的数量，从而让模型变得稀疏，这样可以减少存储和计算。所以优化目标可能是这样的：
$$
\min_{\widehat{W}} \; L(\widehat{W}) \;+\; \lambda \,\|\widehat{W}\|_{0}
$$

问题：$L_0$ 范数是非凸、不连续的，这意味着它没有良好的数学性质，导致优化问题是 NP-hard 的（即计算上非常困难，没有高效的全局最优解算法）。你无法直接使用梯度下降等基于梯度的优化方法来最小化它，因为它的梯度几乎处处为零或不存在。

2. $L_1$ 范数：凸且促进稀疏性
定义：$\|W\|1$ 表示矩阵 $W$ 中所有元素的 绝对值之和：
$$
\|W\|_1 = \sum_{i,j} |w_{ij}|
$$

性质：

凸性：$L_1$ 范数是一个 凸函数。这意味着当我们将它作为正则项加入损失函数时，整个优化问题（如果是凸损失函数）将仍然是凸的，或者至少是更容易优化的。我们可以使用梯度下降及其变种（例如次梯度下降，因为 $|x|$ 在 $x=0$ 处不可导）来找到全局最优解或近似最优解。

促进稀疏性：虽然 $L_1$ 范数不直接统计非零元素的个数，但它有一个非常重要的性质：在优化过程中，为了最小化 $\sum |w_{ij}|$，优化器倾向于将许多权重推向 精确的零，而不是一个非常小但非零的值。这是因为 $|x|$ 在 $x=0$ 附近有一个“尖点”，这使得优化器在接近零时受到更大的惩罚，从而促使权重跨过零点。

3. 为什么是“凸近似”？
近似 $L_0$ 行为：虽然 $L_1$ 不等于 $L_0$，但它能有效地诱导稀疏性，从而在实践中近似达到我们用 $L_0$ 追求的效果（即得到一个有很多零的模型）。
可优化性： $L_1$ 是凸的，这使得优化问题变得可解。它提供了一个数学上更容易处理的替代方案，可以在合理的时间内找到一个好的解。所以，当看到剪枝的优化目标中包含 $L_1$ 正则项时，比如：
$$
\min_{\widehat{W}} \; L(\widehat{W}) + \lambda \|\widehat{W}\|_1
$$
它的意思是：我们希望在保持模型精度的同时（最小化 $L(\widehat{W})$），也能让模型的权重尽可能小，并且倾向于零（最小化 $\lambda |\widehat{W}|_1$），从而达到稀疏化的目的。这种方法是实践中实现模型稀疏化（尤其是非结构化剪枝）非常常见且有效的方式。

---

### 1.4.4 在工程中如何实际使用这些范数？

1. **判断某层是否适合低秩分解**  
   - 计算奇异值 $\{\sigma_i\}$，看前 k 个是否已经占到整体能量的绝大部分：

$$
\frac{\sum_{i=1}^k \sigma_i^2}{\sum_{i=1}^r \sigma_i^2} \approx 0.95 \text{ 或更高}
$$


2. **设计剪枝策略**  
   - 若某层权重的 $L_1$ 分布极不均匀（大量接近 0 的元素），说明存在自然稀疏性 → 可以安全剪枝
   - 可以对每一行/每一列计算 $L_1$ 或 $L_2$ norm，作为“通道重要性”的指标

3. **量化敏感性分析**  
   - 对于谱范数较大的层，可以考虑：
     - 使用更高 bit 的量化（例如 8-bit 而不是 4-bit）
     - 或做更细致的 per-channel scaling 来减小量化误差

> 总结：范数帮助你从“拍脑袋选层/选 rank/选 bit”  
> 变成“有指标、有依据的工程决策”。


### 1.4.5 例子
三类范数的工程决策整合例子。
假设有一个 Transformer FFN 层 𝑊
计算结果如下

| 指标           | 数值           | 解释                |
| ------------ | ------------ | ----------------- |
| 奇异值能量 E(128) | 96.3%        | 很适合低秩分解（rank=128） |
| 平均 L1 norm   | 大量行在 0.01 以下 | 有稀疏性，可做剪枝         |
| 最大奇异值（谱范数）   | 780          | 量化敏感，不要用太低 bit    |

最终工程决策可能是
🔹 第一步：低秩分解

把 4096×4096 降到 rank=128
→ 参数从 16M → 1M
→ FLOPs 从 33M → 2M

🔹 第二步：结构化剪枝

由于 L1 分布稀疏 → 去掉 20% 无用通道
→ 通道数从 4096 → 3276

🔹 第三步：量化

因为谱范数大 → 不能用 INT4
✔ 选 FP16 或 INT8-per-channel

最终效果：

模型大小减少约 16× × 1.25× = 20×

FLOPs 降到原来的 1/10~1/15

延迟显著降低，没有明显精度损失


In [6]:
import torch
import torch.nn as nn
import numpy as np

def get_weight_2d(layer: nn.Module):
    """
    flatten the weight of a layer into 2D matrix
    """
    W = layer.weight.detach().float().clone()

    if isinstance(layer, nn.Linear):
        # [out_features, in_features]
        W2d = W  # already 2D
    elif isinstance(layer, nn.Conv2d):
        # [out_channels, in_channels, kh, kw] -> [out_channels, in_channels*kh*kw]
        out_c, in_c, kh, kw = W.shape
        W2d = W.view(out_c, in_c * kh * kw)
    else:
        raise NotImplementedError(f"Unsupported layer type: {type(layer)}")

    return W2d


def analyze_layer(layer: nn.Module,
                  energy_target: float = 0.98,
                  max_rank_ratio: float = 0.5,
                  max_prune_ratio: float = 0.5):
    """
    for a given layer,
    - SVD analysis → recommend low-rank
    - L1 distribution analysis → recommend pruning ratio (by output channels)
    - Spectral norm analysis → recommend quantization bits
    """
    W2d = get_weight_2d(layer)                  # [out_dim, in_dim]
    out_dim, in_dim = W2d.shape

    # ===== 1. SVD & rank recommendation =====
    # only need singular values, and skip U and V
    # note: for very large matrices, consider using randomized
    sigma = torch.linalg.svdvals(W2d)           # [min(out_dim, in_dim)]
    sigma2 = sigma ** 2
    total_energy = sigma2.sum()
    cum_energy = torch.cumsum(sigma2, dim=0) / total_energy

    # Find the smallest k such that cum_energy[k-1] >= energy_target
    k_candidate = int((cum_energy >= energy_target).nonzero(as_tuple=True)[0][0].item() + 1)
    
    # Do not exceed max_rank_ratio * min_dim to prevent keeping too many components
    max_rank = int(min(out_dim, in_dim) * max_rank_ratio)
    k_recommended = min(k_candidate, max_rank)

    # ===== 2. L1 distribution & pruning ratio recommendation (by output channels) =====
    # compute L1 norm for each output channel
    l1_per_out = W2d.abs().sum(dim=1)          # [out_dim]
    l1_np = l1_per_out.cpu().numpy()
    # compute several percentiles to sense sparsity
    q10, q25, q50 = np.percentile(l1_np, [10, 25, 50])

    # If the gap between the low percentile and the median is large,
    # it indicates that the tail channels are not important and can be pruned.
    # Simple heuristic: check if the 25% percentile is significantly smaller than the 50% percentile.
    prune_ratio = 0.0
    if q25 < 0.3 * q50:
        # very sparse, can prune up to 30%
        prune_ratio = min(0.3, max_prune_ratio)
    elif q25 < 0.5 * q50:
        # slightly sparse, can prune up to 15%
        prune_ratio = min(0.15, max_prune_ratio)
    else:
        prune_ratio = 0.0  # 不建议剪

    # ===== Spectral norm & quantization bits recommendation =====
    spectral_norm = sigma.max().item()

    # Very rough heuristic:
    if spectral_norm < 5:
        quant_bits = 4 
    elif spectral_norm < 20:
        quant_bits = 8 
    else:
        # Recommended minimally 8 bits (or even 16 bits) with per-channel scaling
        quant_bits = 8  

    # print results
    print("=" * 60)
    print(f"Layer: {layer.__class__.__name__}, weight shape = {tuple(layer.weight.shape)}")
    print("--- Low-rank analysis ---")
    print(f"  min(out_dim, in_dim)  = {min(out_dim, in_dim)}")
    print(f"  target energy         = {energy_target:.3f}")
    print(f"  k (reach target)      = {k_candidate}")
    print(f"  k (capped by {max_rank_ratio:.2f}*min_dim) = {k_recommended}")
    print(f"  Compression ratio     ≈ {min(out_dim, in_dim) / k_recommended:.2f}x (in FLOPs/params)")

    print("\n--- L1 sparsity / pruning analysis ---")
    print(f"  L1 per-out quantiles  (10%,25%,50%) = ({q10:.4g}, {q25:.4g}, {q50:.4g})")
    print(f"  Recommended prune ratio (by channels) ≈ {prune_ratio*100:.1f}%")

    print("\n--- Quantization sensitivity ---")
    print(f"  Spectral norm ||W||_2 = {spectral_norm:.4g}")
    print(f"  Recommended quant bits = {quant_bits}-bit")

    return {
        "sigma": sigma,
        "cum_energy": cum_energy,
        "recommended_rank": k_recommended,
        "recommended_prune_ratio": prune_ratio,
        "spectral_norm": spectral_norm,
        "recommended_quant_bits": quant_bits,
    }


try:
    from torchvision.models import resnet18
    # load no pre-trained weights
    model = resnet18(weights=None)  
except Exception as e:
    raise e

torch.manual_seed(0)

# Traverse all sub-modules
for name, module in model.named_modules():
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        print(f"\n>>> Analyzing layer: {name}")
        analyze_layer(module, energy_target=0.98, max_rank_ratio=0.5, max_prune_ratio=0.5)



>>> Analyzing layer: conv1
Layer: Conv2d, weight shape = (64, 3, 7, 7)
--- Low-rank analysis ---
  min(out_dim, in_dim)  = 64
  target energy         = 0.980
  k (reach target)      = 58
  k (capped by 0.50*min_dim) = 32
  Compression ratio     ≈ 2.00x (in FLOPs/params)

--- L1 sparsity / pruning analysis ---
  L1 per-out quantiles  (10%,25%,50%) = (2.725, 2.836, 2.943)
  Recommended prune ratio (by channels) ≈ 0.0%

--- Quantization sensitivity ---
  Spectral norm ||W||_2 = 0.5254
  Recommended quant bits = 4-bit

>>> Analyzing layer: layer1.0.conv1
Layer: Conv2d, weight shape = (64, 64, 3, 3)
--- Low-rank analysis ---
  min(out_dim, in_dim)  = 64
  target energy         = 0.980
  k (reach target)      = 62
  k (capped by 0.50*min_dim) = 32
  Compression ratio     ≈ 2.00x (in FLOPs/params)

--- L1 sparsity / pruning analysis ---
  L1 per-out quantiles  (10%,25%,50%) = (25.9, 26.52, 27.24)
  Recommended prune ratio (by channels) ≈ 0.0%

--- Quantization sensitivity ---
  Spectral norm

## 1.5 卷积的线性代数表达：Conv = MatMul

卷积层（Conv2d）是 CNN 和很多视觉模型的核心。  
从数学上，它是一个带有结构的线性算子：

$$
y = K * x
$$


- $x$：输入特征图（如 $\mathbb{R}^{C_{\text{in}} \times H \times W}$）  
- $K$：卷积核（如 $\mathbb{R}^{C_{\text{out}} \times C_{\text{in}} \times k_h \times k_w}$）

通过适当展开和重排，可以把它改写为**矩阵乘法**。

---

### 1.5.1 im2col：把局部 patch 展成列向量

以单通道、kernel=2×2、stride=1 为例：

输入：

```text
1 2 3
4 5 6
7 8 9
```

卷积核：

```text
k11 k12
k21 k22
```

所有 2×2 patch：

1. 上左：$\{1,2,4,5\}$
2. 上右：$\{2,3,5,6\}$
3. 下左：$\{4,5,7,8\}$
4. 下右：$\{5,6,8,9\}$

我们把每个 patch 按固定顺序展开成列向量（也就是从左上开始，窗口大小 2×2，水平/垂直各滑动一步，枚举所有可能的 2×2 patch）：

$$
x_1 = [1,2,4,5]^\top, \quad
x_2 = [2,3,5,6]^\top, \quad
x_3 = [4,5,7,8]^\top, \quad
x_4 = [5,6,8,9]^\top
$$


然后组成一个矩阵 $X_{\text{col}} = \{x_1, x_2, x_3, x_4\}$：

$$
X_{\text{col}} =
\begin{bmatrix}
1 & 2 & 4 & 5 \\
2 & 3 & 5 & 6 \\
4 & 5 & 7 & 8 \\
5 & 6 & 8 & 9
\end{bmatrix}
\in \mathbb{R}^{4 \times 4}
$$


卷积核也展开成向量：

$$
w = [k_{11}, k_{12}, k_{21}, k_{22}]
$$


则每个输出位置：

$$
y_i = w x_i
$$


所有位置一起写成矩阵形式：

$$
y^\top = w X_{\text{col}}
$$


---

### 1.5.2 多通道 / 多卷积核的情况

对于一般情况：

- 输入：$x \in \mathbb{R}^{C_{\text{in}} \times H \times W}$
- 卷积核：$K \in \mathbb{R}^{C_{\text{out}} \times C_{\text{in}} \times k_h \times k_w}$

通过 im2col：

- 生成矩阵 $X_{\text{col}} \in \mathbb{R}^{(C_{\text{in}} k_h k_w) \times L}$  
  其中 L 是滑窗数量（输出空间尺寸的乘积）
- 卷积核压平成矩阵 $W \in \mathbb{R}^{C_{\text{out}} \times (C_{\text{in}} k_h k_w)}$

则卷积可以写成：

$$
Y = W X_{\text{col}}
$$


- $Y \in \mathbb{R}^{C_{\text{out}} \times L}$，之后 reshape 回输出尺寸

**【核心结论】**  
> Conv2d 经过 im2col 变换后，本质就是一个 GEMM。

---
### 1.5.3 多核多通道例子
设：
- 输入特征图：$x \in \mathbb{R}^{C_{\text{in}} \times H \times W},\quad
C_{\text{in}} = 2,\ H=W=3$
- 卷积核： $K \in \mathbb{R}^{C_{\text{out}} \times C_{\text{in}} \times k_h \times k_w},\quad
C_{\text{out}} = 2,\ k_h=k_w=2$
- stride = 1, padding = 0
- 输出空间尺寸：$H_{\text{out}} = W_{\text{out}} = 3-2+1 = 2$
所以一共 $L = H_{\text{out}} \cdot W_{\text{out}} = 4$ 个输出位置。
---
1. 输入数据（2 个通道）
- 第 1 个通道（c=0）：
$\begin{array}{c} x^{(0)} =
\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix} \end{array}$

- 第 2 个通道（c=1）：
$\begin{array}{c} x^{(1)} =
\begin{bmatrix}
10 & 11 & 12 \\
13 & 14 & 15 \\
16 & 17 & 18
\end{bmatrix} \end{array}$
---
2. 卷积核（2 个输出通道 = 2 个 filter）
每个卷积核对两个输入通道都有自己的 2×2 kernel：
- 第 1 个输出通道（filter 0）：
$\begin{array}{c} K^{(0,0)} =
\begin{bmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}
\end{bmatrix},
\quad
K^{(0,1)} =
\begin{bmatrix}
b_{11} & b_{12} \\
b_{21} & b_{22}
\end{bmatrix} \end{array}$

- 第 2 个输出通道（filter 1）：
$\begin{array}{c} K^{(1,0)} =
\begin{bmatrix}
c_{11} & c_{12} \\
c_{21} & c_{22}
\end{bmatrix},
\quad
K^{(1,1)} =
\begin{bmatrix}
d_{11} & d_{12} \\
d_{21} & d_{22}
\end{bmatrix} \end{array}$

这里 $(0,0)$  表示：输出通道 0，对输入通道 0 的 kernel， $(0,1)$ 表示：输出通道 0，对输入通道 1 的 kernel，依此类推。

---

3. 用 im2col 展开多通道 patch → 列向量
输出空间是 2×2，所以有 4 个滑窗位置：

- 左上：覆盖输入坐标 (0:2, 0:2)

- 右上：覆盖 (0:2, 1:3)

- 左下：覆盖 (1:3, 0:2)

- 右下：覆盖 (1:3, 1:3)

对每个位置，我们要从 两个通道 各取一个 2×2 patch，然后全部摊平接在一起，形成一个长度为 $C_{\text{in}} \cdot k_h \cdot k_w = 2 \cdot 2 \cdot 2 = 8$ 的列向量。

🔹 位置 1：左上 patch（top-left）
- 通道 0 的 2×2 patch：
$\begin{bmatrix}
1 & 2 \\
4 & 5
\end{bmatrix}$
- 通道 1 的 2×2 patch：
$\begin{bmatrix}
10 & 11 \\
13 & 14
\end{bmatrix}$

按固定顺序（比如：通道顺序优先，每个 patch 按行展平）摊平并拼接：
$\begin{array}{c} x_{\text{col},1} =
\begin{bmatrix}
1 \\ 2 \\ 4 \\ 5 \\ 10 \\ 11 \\ 13 \\ 14
\end{bmatrix}
\in \mathbb{R}^{8} \end{array}$

🔹 位置 2：右上 patch（top-right）
- 通道 0：$\begin{bmatrix} 2 & 3 \\ 5 & 6 \end{bmatrix}$
- 通道 1：$\begin{bmatrix} 11 & 12 \\ 14 & 15 \end{bmatrix}$
展开：
$\begin{array}{c} x_{\text{col},2} =
\begin{bmatrix}
2 \\ 3 \\ 5 \\ 6 \\ 11 \\ 12 \\ 14 \\ 15
\end{bmatrix} \end{array}$

🔹 位置 3：左下 patch（bottom-left）
- 通道 0：$\begin{bmatrix} 4 & 5 \\ 7 & 8 \end{bmatrix}$
- 通道 1：$\begin{bmatrix} 13 & 14 \\ 16 & 17 \end{bmatrix}$
展开：
$\begin{array}{c} x_{\text{col},3} =
\begin{bmatrix}
4 \\ 5 \\ 7 \\ 8 \\ 13 \\ 14 \\ 16 \\ 17
\end{bmatrix} \end{array}$

🔹位置 4：右下 patch（bottom-right）
- 通道 0：$\begin{bmatrix} 5 & 6 \\ 8 & 9 \end{bmatrix}$
- 通道 1：$\begin{bmatrix} 14 & 15 \\ 17 & 18 \end{bmatrix}$
展开：
$\begin{array}{c} x_{\text{col},4} =
\begin{bmatrix}
5 \\ 6 \\ 8 \\ 9 \\ 14 \\ 15 \\ 17 \\ 18
\end{bmatrix} \end{array}$

🔹 组合成 $X_{\text{col}}$ 把 4 个列向量拼在一起：
$\begin{array}{c} X_{\text{col}} =
\begin{bmatrix}
1  & 2  & 4  & 5  \\
2  & 3  & 5  & 6  \\
4  & 5  & 7  & 8  \\
5  & 6  & 8  & 9  \\
10 & 11 & 13 & 14 \\
11 & 12 & 14 & 15 \\
13 & 14 & 16 & 17 \\
14 & 15 & 17 & 18
\end{bmatrix}
\in \mathbb{R}^{8 \times 4} \end{array}$

---
4。 卷积核展平成矩阵 W（多卷积核 → 多行）
每个输出通道的 filter 是 两个 2×2 的 kernel（对应 2 个输入通道），我们也把它们展平并拼接：
- 对输出通道 0（filter 0）：
$\begin{array}{c} w^{(0)} =
\begin{bmatrix}
a_{11} \\ a_{12} \\ a_{21} \\ a_{22} \\
b_{11} \\ b_{12} \\ b_{21} \\ b_{22}
\end{bmatrix}^\top \end{array}$	​
- 对输出通道 1（filter 1）：
$\begin{array}{c} w^{(1)} =
\begin{bmatrix}
c_{11} \\ c_{12} \\ c_{21} \\ c_{22} \\
d_{11} \\ d_{12} \\ d_{21} \\ d_{22}
\end{bmatrix}^\top \end{array}$

- 这样组合成矩阵$W$：
$\begin{array}{c} W =
\begin{bmatrix}
w^{(0)} \\
w^{(1)}
\end{bmatrix}
\in \mathbb{R}^{2 \times 8} \end{array}$
---

5. 统一成一个矩阵乘法：Y = W X_col
现在卷积运算可以写成：$Y = W X_{\text{col}}$

- $W \in \mathbb{R}^{C_{\text{out}} \times (C_{\text{in}} k_h k_w)} = \mathbb{R}^{2 \times 8}$
- $X_{\text{col}} \in \mathbb{R}^{(C_{\text{in}} k_h k_w) \times L} = \mathbb{R}^{8 \times 4}$
- $Y \in \mathbb{R}^{C_{\text{out}} \times L} = \mathbb{R}^{2 \times 4}$
其中：
- 第 1 行对应输出通道 0 在所有 4 个空间位置的输出
- 第 2 行对应输出通道 1
- 再把每一行 reshape 回 2×2，就得到标准的 2×2 feature map：$Y^{(c)} \in \mathbb{R}^{2 \times 2},\quad c=0,1$

### 1.5.4 在推理加速中的意义

1. **重用高性能 GEMM 内核**  
   - 不需要为每种卷积形状写全新的 kernel  
   - 只需实现 im2col（或更高级的变换）+ 调用 GEMM
2. **统一优化目标**  
   - GEMM 的 tiling、向量化、cache 利用一旦写好，Conv 也能享受
3. **更容易做量化/剪枝**  
   - 权重在 im2col 形式下就是一个矩阵 W，方便 per-channel / per-row 量化和剪枝

> 在高性能推理库中，Conv2d 的实现通常要么：  
> - 走 im2col + GEMM 路线，  
> - 要么走专门的 convolution kernel，但其内部仍然围绕“块矩阵乘法”的思想设计。

---




In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

# ======================
# 1. 定义一个简单的 Conv2d
# ======================
in_channels = 2
out_channels = 3
kernel_size = 3
stride = 1
padding = 1

conv = nn.Conv2d(
    in_channels=in_channels,
    out_channels=out_channels,
    kernel_size=kernel_size,
    stride=stride,
    padding=padding,
    bias=True,
)

# 构造输入：batch_size=1, C=2, H=W=4
x = torch.randn(1, in_channels, 4, 4)

# 用 nn.Conv2d 直接算一遍（标准实现）
y_conv = conv(x)  # 形状 [1, out_channels, H_out, W_out]
print("y_conv shape:", y_conv.shape)

# ======================================
# 2. 用 im2col + matmul 手工实现同样的卷积
# ======================================

# unfold 相当于 im2col：
# x_unfold: [N, C_in * k_h * k_w, L]
# 其中 L = H_out * W_out
x_unfold = F.unfold(
    x,
    kernel_size=kernel_size,
    dilation=1,
    padding=padding,
    stride=stride,
)
# 取一下展开后的尺寸
N, K, L = x_unfold.shape  # K = C_in * k_h * k_w
print("x_unfold shape:", x_unfold.shape)  # [N, K, L]

# 卷积核权重： [C_out, C_in, k_h, k_w] -> [C_out, K]
W = conv.weight.view(out_channels, -1)  # [C_out, C_in * k_h * k_w]
b = conv.bias                           # [C_out]

# 对每个 batch 独立做矩阵乘法：
# 对某个样本 n：
#   x_unfold[n]: [K, L]
#   输出 feature: [C_out, L] = W @ x_unfold[n] + b
y_im2col_list = []
for n in range(N):
    # [C_out, L] = [C_out, K] @ [K, L]
    y_n = W @ x_unfold[n]  # [C_out, L]
    # 加 bias：在第 0 维 broadcast
    y_n = y_n + b.view(-1, 1)  # [C_out, L]
    y_im2col_list.append(y_n)

# 堆回 batch 维： [N, C_out, L]
y_im2col = torch.stack(y_im2col_list, dim=0)

# 将 L 展回 H_out, W_out
H_out = y_conv.shape[2]
W_out = y_conv.shape[3]
y_im2col = y_im2col.view(N, out_channels, H_out, W_out)

print("y_im2col shape:", y_im2col.shape)

# ============================
# 3. 对比 nn.Conv2d 与 im2col+matmul
# ============================

diff = (y_conv - y_im2col).abs().max().item()
print("max |y_conv - y_im2col| =", diff)

if diff < 1e-5:
    print("✅ Conv2d == im2col + matmul (within numerical tolerance)")
else:
    print("⚠️ there is a discrepancy between Conv2d and im2col + matmul!")


y_conv shape: torch.Size([1, 3, 4, 4])
x_unfold shape: torch.Size([1, 18, 16])
y_im2col shape: torch.Size([1, 3, 4, 4])
max |y_conv - y_im2col| = 0.0
✅ Conv2d == im2col + matmul (within numerical tolerance)


## 1.6 Attention 的线性代数结构

以单头 self-attention 为例（不考虑 bias & mask）：

$$
Q = X W_Q,\quad
K = X W_K,\quad
V = X W_V \\
A = \text{softmax}\left(\frac{Q K^\top}{\sqrt{d_k}}\right) \\
O = A V
$$


- $X \in \mathbb{R}^{T \times d_{\text{model}}}$：输入序列
- $W_Q, W_K, W_V \in \mathbb{R}^{d_{\text{model}} \times d_k}$
- $Q,K,V \in \mathbb{R}^{T \times d_k}$
- $A \in \mathbb{R}^{T \times T}$
- $O \in \mathbb{R}^{T \times d_v}$

其中：

1. $Q = X W_Q$ 是一个 GEMM（$T \times d_{\text{model}}$ 乘 $d_{\text{model}} \times d_k$）
2. $K = X W_K$、$V = X W_V$ 同理
3. $QK^\top$ 是一个 **非常大的 GEMM**（$T \times d_k$ 乘 $d_k \times T$）
4. $A V$ 又是一个 GEMM（$T \times T$ 乘 $T \times d_v$）
- ${d_{\text{model}}}$: 模型的隐藏维度，每个 token 的向量长度（Transformer 主通道维度）。
- ${d_k}$: 单个注意力头的 Query/Key 投影维度（通常 ${d_{\text{model}}}$ / num_heads）。
- ${d_v}$: Value 投影维度（常设成和 ${d_k}$ 相等）。
- ${T}$: 序列长度（时间步数/ token 数


**【工程视角】**  
- Attention 中真正“贵”的地方就是两个矩阵乘法：$QK^\top$ 和 $A V$
- softmax 本身 FLOPs 不多，但存在数值稳定性的挑战（第 8 章展开）
- 所有对 Attention 的加速（如 FlashAttention、各种 kernel fusion）本质都是：
  - 减少显存读写
  - 改善访问模式
  - 在不显式构造 $T \times T$ 矩阵的前提下，实现等价的线性代数运算

> 一旦站在“线性代数”视角看 Attention，就更容易理解各种加速论文的思路。

---

## 1.7 Embedding：查表 = 选行 = 稀疏 GEMM

Embedding 矩阵：$E \in \mathbb{R}^{V \times D}$

- V：词表大小
- D：隐层维度 / embedding 维度

给定 token id = i，Embedding 的作用就是：

$$
\text{Embedding}(i) = E_i
$$


即：**从矩阵 E 中选取第 i 行**。

如果构造一个 one-hot 向量 $x \in \mathbb{R}^V$：

$$
x_j = \begin{cases}
1, & j = i \\
0, & \text{else}
\end{cases}
$$


则：

$$
y = x^\top E = E_i
$$


也就是说：

> Embedding 在数学上等价于：**one-hot 向量乘 embedding 矩阵的矩阵乘法（一个极其稀疏的 GEMM）**。

**【工程视角】**  

- 对 Embedding 来说，算力（FMA）几乎不构成瓶颈，因为没有大规模乘加
- 真正的瓶颈是：
  - 内存访问（随机访问 E 的不同行）
  - cache 命中率
  - 带宽

这解释了为什么：  
> 巨大词表的 LLM，在部署时经常是 “embedding 和 LM head 很吃带宽”；  
> 而中间层（GEMM）则更吃计算。

---

## 1.8 数据布局（Memory Layout）与线性代数

矩阵在内存中可以有两种常见布局：

- row-major（行优先，C-style）
- column-major（列优先，Fortran/BLAS-style）

附：
BLAS（Basic Linear Algebra Subprograms）是密集线性代数的标准 API 规范，一组基础算子库接口：

- Level 1/2/3：向量运算、矩阵‑向量、矩阵‑矩阵（GEMM 在 Level 3）。
- 不同平台有不同实现：OpenBLAS/BLIS、MKL、Apple Accelerate、cuBLAS 等，提供高度优化的内核。

对于 $A \in \mathbb{R}^{M \times K}$：

- row-major：同一行的元素连续存放
- column-major：同一列的元素连续存放

这会影响：

- 连续访问的方向（stride=1）
- 缓存局部性（cache locality）
- 是否容易被 SIMD 向量化

**【工程视角】**  

- 很多 GEMM 内核会要求 A/B/C 矩阵采用特定布局（如 A row-major，B col-major），以便：
  - 在 inner-k loop 中连续访问数据
  - 利用硬件的向量加载指令（如 `ld1q`、`vmovaps` 等）
- 这就是为什么你在 kernel 代码中会看到大量 transpose / layout transform，它们本质是为线性代数运算创建“更友好的 memory layout”。

---

## 1.9 小结：线性代数是 Runtime Inference 的共同语言

本章核心结论：

1. **推理的大部分 FLOPs 都来自 GEMM（矩阵乘法）**  
   - 全连接 / MLP / Attention projection / Conv2d (im2col) 等
2. **低秩近似（SVD）是最重要的压缩数学之一**  
   - 通过 rank-k 近似，把大矩阵拆成两个小矩阵乘法
3. **矩阵范数给出度量压缩误差的工具**  
   - Frobenius、谱范数、L1/L0
4. **Conv、Attention、Embedding 都可以在“矩阵视角”下统一看待**
5. **数据布局与线性代数紧密相关**  
   - Row-major / col-major / tiling 决定是否能高效利用硬件

> 接下来几章会在这个基础上，继续引入：  
> - 为何这些矩阵要被“优化”（数值优化章节）  
> - 为何我们敢于用粗糙的近似（近似理论章节）  
> - 如何在概率/信息论框架下理解压缩与量化  
> - 如何结合硬件特性真正做到“跑满”推理芯片

# 第 2 章：数值优化（Optimization for Pruning, Quantization & Distillation）

本章目标：理解剪枝、量化、蒸馏等操作背后的“优化问题”视角。  
你不需要成为优化理论专家，但需要：

- 看懂常见损失函数与正则项的形式
- 理解 L0/L1 稀疏化、量化参数优化的大致思路
- 知道多目标优化（精度 vs 延迟）的典型写法

---

## 2.1 剪枝（Pruning）的优化视角

### 2.1.1 基本目标

给定训练好的权重 $W$、损失函数 $L(W)$，剪枝希望：

- 大量元素变为 0（稀疏）
- 损失/精度变化尽量小

可以形式化为：

$$
\min_{\hat{W}} L(\hat{W}) + \lambda \|\hat{W}\|_0
$$


其中：
- 这里 $L$ 是模型的损失函数（训练时用的目标函数），在权重为 $\hat{W}$ 时的损失值
- $\|\hat{W}\|_0$：非零元素个数（稀疏性度量）
- $\lambda$：权衡“精度 vs 稀疏度”的超参数

这个问题一般是 NP-hard，常见近似方式有：

1. 用 $L_1$ 范数替代 $L_0$：

$$
\min_{\hat{W}} L(\hat{W}) + \lambda \|\hat{W}\|_1
$$


2. 先正常训练，再做基于某种 saliency 的后处理剪枝：
   - 按权重绝对值大小剪掉小值
   - 按 Hessian 近似（如 OBS/OBD）计算敏感度

---

### 2.1.2  Hessian矩阵 & saliency score
后训练剪枝（post-training pruning）核心是

- 这条边（某个 weight）删掉会不会伤害 loss？

- 哪个通道更重要？

- 哪层该多剪？哪层不能乱动？j z

🔷 1. 什么是 Hessian？
Hessian 是 loss 对参数的二阶导数矩阵：
$$H = \nabla^2_{W} L(W)$$
更精确地：
- 如果模型参数是$$W \in \mathbb{R}^n$$， 则 Hessian 是：$$H \in \mathbb{R}^{n \times n}$$
其中每个元素： $$H_{ij} = \frac{\partial^2 L}{\partial W_i \partial W_j}$$
代表 loss 对两个 weight 的二阶相互影响。

🔷 2. 为什么 Hessian 与剪枝有关？

问题：
如果把某个 weight从 $w_i$->0, loss 会变多少？
如果你直接看$|w_i|$,只能知道它“大小”，不知道：
- 这个权重对 loss 的敏感度如何

- 换句话说，它是不是“关键路径”上的重要参数？

简单绝对值是**粗糙代理**。Hessian 则衡量了：**改变一个权重，会让 loss 增大多少**, 更精确。

🔥 3. 用二阶泰勒展开估计“剪掉某个 weight 会带来多少损失增加”？
假设我们要把第$i$个 weight从当前$w_i$设置成 0：
$$\Delta w_i = -w_i$$
loss 的变化（用二阶泰勒展开）：
$$\Delta L \approx 
\frac{\partial L}{\partial w_i}\Delta w_i + \frac{1}{2} H_{ii} (\Delta w_i)^2$$
但：
- 在训练完的点, 梯度$\partial L/\partial w_i \approx 0$（因为训练收敛）
- 所以剩下：$$\Delta L \approx \frac{1}{2} H_{ii} w_i^2$$

至此 **saliency score** = $H_{ii} w_i^2$
这比单看$|w_i|$更有意义

🔷 直觉的解释
- 泰勒展开说：
“在一个点附近，函数的变化 ≈ 一阶（斜率） + 二阶（弯曲程度）。”

- 在已经训练好的点：
一阶导（斜率）接近 0，所以变化主要靠二阶导决定。

- 对于剪枝：
你把$w_i$改成 0，相当于在参数空间里沿着第 i 个坐标方向，走了一个步长 $-w_i$
损失增加 ≈ 弯曲程度 × 你走的距离的平方。
	​
- “弯曲程度”在这个方向就是 Hessian 的对角线元素 $H_{ii}$
 - 如果这个方向上弯得很厉害（H 大），说明对这个参数很敏感，不能乱剪。
 - 如果这个方向几乎是平的（H 小），即使 $w_i$不为零，也可以大胆剪

所以：
- 只看$|w_i|$ → 只是看这个参数“大不大”。
- 看$H_{ii} w_i^2$ → 看“删掉它造成 loss 上升有多大”，这是更合理的“重要性”度量。

🔷 4. OBS（Optimal Brain Surgeon）与 OBD（Optimal Brain Damage）
这是 1990 年代的经典剪枝方法：

🟦 OBD（Optimal Brain Damage）

它使用对角 Hessian：
$$\text{saliency}_i = \frac{1}{2} H_{ii} w_i^2$$
直觉：
* 如果 $H_{ii}$很大 → 该weight 非常敏感
* 就算它很小，也不能轻易删
* 如果 $H_{ii}$ 很小 → 这个 weight 不重要
* 可以安全剪掉

缺点：忽略 Hessian 的 off-diagonal（跨 weight 相关性）。

🟩 OBS（Optimal Brain Surgeon）
OBS 更精确：
$$\text{saliency}_i = 
\frac{1}{2} \frac{w_i^2}{(H^{-1})_{ii}}$$

OBS 使用逆 Hessian，考虑所有 weight 之间的相互影响，理论上更准确：

- 如果 weight 能被其他参数补偿（Hessian inverse 反映相关性），它更适合删
- 但计算 Hessian inverse 太贵，现代很少用全量 OBS。

🔥 5. 工程做法（Pruning-aware methods）
真实工程中，我们不可能算真正的 Hessian（太大），但可以：

✔ 近似对角线：
- 用 Fisher 信息矩阵对角线
- 用 Hutchinson estimator
- 用梯度平方的 moving average , 类似 Adam 的 $v_t$ ，这就把 Hessian 估计变成可训练、可部署的东西。
此处
$H_{ii} \approx \mathbb{E}[g_i^2]$， 

$g_i = \frac{\partial L}{\partial w_i}$
这个梯度随着 batch 而变化，因为 loss 是基于 mini-batch 求出来的，所以通常用：
$$\mathbb{E}[g_i^2] = \text{over minibatches }$$
换句话说：

- 每个 batch 会算一次梯度 $g = \nabla L$
- 取它的平方 $g_i^2$
- 再对 batch 求期望（平均）

就得到了对角线 Hessian 的估计。

📌 为什么梯度平方可以近似 Hessian 对角线？
在深度学习中，尤其是使用 **Cross-Entropy loss + Softmax** 的模型，（其他的不一定对，比如MSE（平方误差回归））里：
$$H_{ii} \approx \mathbb{E}[g_i^2]$$
原因是：
1. Hessian = 二阶变化率；Fisher 信息 = 梯度方差
对数似然条件下，有数学定理（Cramer–Rao + Information Equality）：
$$\mathbb{E}[ \nabla^2 L ] = \mathbb{E}[ g g^\top ]$$
取对角线：
$$H_{ii} \approx \mathbb{E}[g_i^2]$$
换句话说：
**梯度的方差 = 二阶曲率（Hessian 对角线**

📌 工程上如何计算$E[g_i^2]$?
```code
hessian_diag[i] += grad[i]**2
count += 1
...
hessian_diag /= count
```
这就是 Hessian 对角线的无偏估计。

📌 直觉的解释
- 梯度是“损失对参数的一阶反应”

- 梯度平方的期望就是“损失在这个方向上的波动强度”

- 波动强说明“这个方向上 loss 很陡” → Hessian 大

- 波动弱说明“这个方向平缓” → Hessian 小
所以：
$H_{ii} \approx E[g_i^2]$
---

#### 2.1.2.1 二阶泰勒展开详细推导
现在参数是向量： $W = (w_1, w_2, \dots, w_n)^\top$
loss 是 $L(W)$
在点 𝑊 附近做二阶泰勒展开：
$$L(W + \Delta W)
\approx
L(W)
+ \nabla L(W)^\top \Delta W
+ \frac{1}{2} \Delta W^\top H \Delta W$$

这里：
- $\nabla L(W)$是梯度向量（长度 n）
- $H = \nabla^2 L(W)$ 是 Hessian 矩阵（n×n

现在 **我们只动第 i 个参数，把它变成 0**
也就是说：
- 原来：$w_i$
- 剪枝后：$w_i^{\text{new}} = 0$
所以：
$$\Delta W = W_{\text{new}} - W_{\text{old}}
= (0,\dots,0, -w_i, 0,\dots,0)^\top
= -w_i e_i$$

代入泰勒展开公式
$$\Delta L
= L(W + \Delta W) - L(W)
\approx \nabla L(W)^\top \Delta W + \frac{1}{2} \Delta W^\top H \Delta W$$

梯度向量：
$\begin{array}{c} \nabla L(W) = 
\begin{bmatrix}
\frac{\partial L}{\partial w_1} \\
\frac{\partial L}{\partial w_2} \\
\vdots \\
\frac{\partial L}{\partial w_n}
\end{bmatrix} \end{array}$


把 $\Delta W = -w_i e_i$代进去：
- 一阶项：
$$\nabla L(W)^\top \Delta W
= \nabla L(W)^\top (-w_i e_i)
= -w_i \frac{\partial L}{\partial w_i}$$

- 二阶项
$$\Delta W^\top H \Delta W
= (-w_i e_i)^\top H (-w_i e_i)
= w_i^2 \, e_i^\top H e_i$$
但：
$$e_i^\top H e_i = H_{ii}$$
所以：
$$\Delta W^\top H \Delta W = w_i^2 H_{ii}$$
于是：
$$\Delta L \approx -w_i \frac{\partial L}{\partial w_i}
+ \frac{1}{2} w_i^2 H_{ii}$$

**再次使用“在收敛点梯度 ≈ 0”**, 在训练好的参数$W^\star$附近：
$$\frac{\partial L}{\partial w_i}(W^\star) \approx 0$$
于是：
$$\Delta L \approx \frac{1}{2} H_{ii} w_i^2$$
这就是**OBD（Optimal Brain Damage）中的 saliency**: $\text{saliency}_i = \frac{1}{2} H_{ii} w_i^2$


#### 2.1.2.2 pytorch CrossEntropy 下的 Hessian 对角线 vs E[g²]

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd.functional import hessian

torch.manual_seed(0)

# ===========================
# 1. 构造一个小的 logistic 回归模型
#    y ∈ {0,1}, p = sigmoid(xW + b)
# ===========================

in_dim = 2

N = 500
X = torch.randn(N, in_dim)

# 真实参数（只用于生成数据）
true_theta = torch.tensor([2.0, -3.0, 0.5])  # [w1, w2, b]
true_W = true_theta[:in_dim].view(in_dim, 1) # [2,1]
true_b = true_theta[in_dim:].view(1)         # [1]

logits = X @ true_W + true_b            # [N,1]
probs = torch.sigmoid(logits)           # [N,1]
y = torch.bernoulli(probs).view(-1).long()  # 转成 int labels [N]

# 我们同样用一维参数向量 theta = [w1, w2, b] 来优化
theta = torch.randn(in_dim + 1, requires_grad=True)

def unpack_theta(th):
    W = th[:in_dim].view(in_dim, 1)   # [2,1]
    b = th[in_dim:].view(1)           # [1]
    return W, b

def loss_fn(theta, X, y):
    W, b = unpack_theta(theta)        # W: [2,1], b: [1]
    logits = X @ W + b                # [N,1]
    logits = logits.view(-1, 1)
    # Binary cross entropy with logits
    loss = F.binary_cross_entropy_with_logits(logits, y.float().view(-1, 1))
    return loss

# 简单训练一下，让 theta 接近“收敛点”
optim = torch.optim.SGD([theta], lr=0.1)

for step in range(300):
    optim.zero_grad()
    loss = loss_fn(theta, X, y)
    loss.backward()
    optim.step()

print("Trained loss:", loss_fn(theta, X, y).item())
print("Trained theta:", theta.detach().numpy())

# ==========================================
# 2. 计算“真·Hessian 对角线”：H_ii
# ==========================================

def loss_only_theta(th):
    return loss_fn(th, X, y)

H = hessian(loss_only_theta, theta)   # [P,P], P = in_dim+1=3
H_diag_true = torch.diag(H).detach()

print("\nTrue Hessian diagonal H_ii:")
print(H_diag_true)

# ==========================================
# 3. 用多 batch 的梯度平方平均 E[g^2] 近似 H_ii
# ==========================================

num_batches = 100
batch_size = 64

g2_accum = torch.zeros_like(theta)

for _ in range(num_batches):
    idx = torch.randint(0, N, (batch_size,))
    Xb = X[idx]
    yb = y[idx]

    loss_b = loss_fn(theta, Xb, yb)
    grad_theta = torch.autograd.grad(loss_b, theta, retain_graph=False, create_graph=False)[0]

    g2_accum += grad_theta.detach() ** 2

H_diag_est = g2_accum / num_batches

print("\nEstimated Hessian diagonal via E[g^2]:")
print(H_diag_est)

# ==========================================
# 4. 对比两者
# ==========================================

abs_diff = (H_diag_true - H_diag_est).abs()
rel_diff = abs_diff / (H_diag_true.abs() + 1e-8)

print("\nAbsolute diff |H_ii - E[g_i^2]|:")
print(abs_diff)

print("\nRelative diff |H_ii - E[g_i^2]| / |H_ii|:")
print(rel_diff)

Trained loss: 0.33923059701919556
Trained theta: [ 1.3182789 -1.9975102  0.4114626]

True Hessian diagonal H_ii:
tensor([0.0965, 0.0759, 0.1324])

Estimated Hessian diagonal via E[g^2]:
tensor([0.0015, 0.0012, 0.0014])

Absolute diff |H_ii - E[g_i^2]|:
tensor([0.0949, 0.0747, 0.1310])

Relative diff |H_ii - E[g_i^2]| / |H_ii|:
tensor([0.9839, 0.9840, 0.9891])


### 2.1.3 补：Fisher 信息矩阵
Fisher 信息矩阵（Fisher Information Matrix, FIM） 衡量的是：

模型对参数变化的敏感度

或

用参数去解释数据时，不确定性有多小

“信息越大 → 参数越重要，不能动”
“信息越小 → 参数不重要，可以剪 / 量化得更激进”

---
1. Fisher 信息矩阵的正式定义
给定模型的对数似然（log-likelihood）：$\log p_\theta(x)$

Fisher 信息矩阵定义为：
$$F(\theta) = \mathbb{E}\left[ \left( \nabla_\theta \log p_\theta(x) \right)
       \left( \nabla_\theta \log p_\theta(x) \right)^\top \right]$$

也就是说：

- 先对 log-likelihood 求梯度（score function）

- 再求它与自身的外积

- 再在数据分布上取期望

---
2. 深度学习里更常用的表达
当我们用 CrossEntropy Loss 时：
$$L(\theta) = -\log p_\theta(y|x)$$
梯度就是：
$$g = \nabla_\theta L = - \nabla_\theta \log p_\theta(y|x)$$
于是 Fisher 信息矩阵等价于：
$$F(\theta) = \mathbb{E}[g\, g^\top]$$

对角线就是：
$$F_{ii} = \mathbb{E}[g_i^2]$$
也就是
$$H_{ii} \approx \mathbb{E}[g_i^2]$$

---
3. 为什么 Fisher 信息矩阵在深度学习中很重要？
Fisher 信息告诉我们：

如果参数 θ 发生微小变化，模型预测会有多大波动？

换句话说：
- $F_{ii}$ 大 → 参数非常“敏感” → 很重要 → 不宜剪
- $F_{ii}$小 → 参数基本不起作用 → 可以剪 / 可以低 bit 量化 

所以：

✔ Fisher 信息可用于剪枝（pruning saliency）

✔ 可用于量化（quantization sensitivity）

✔ 可用于确定哪些层的 rank 要下调（low-rank SVD）

✔ 可用于 LayerNorm / Whitening 的稳定性分析


### 2.1.4 结构化剪枝（channel/head pruning）

可以把通道剪枝写成：

$$
\min_{\theta, m} \ L(\theta \odot m) + \lambda \|m\|_0
$$


- $\theta$：原始参数
- $m$：mask，按通道/块为单位取 0 或 1
- $\odot$：逐元素或逐通道乘法

通常会：

1. 训练一个带有可微近似 mask 的模型（如用 sigmoid/Concrete distribution）
2. 在训练过程中逐渐压缩某些通道的权重
3. 最后将接近 0 的通道硬剪掉

> 本质仍然是“带稀疏约束的优化问题”，只是作用对象从单个元素提升到了“结构化块”。

---


## 2.2 量化（Quantization）的优化视角

### 2.2.1 简单的 uniform quantization

考虑对权重集合 $\{w_i\}$ 做对称均匀量化，bit 宽为 b：

- 量化级别：$q_i \in \{-Q,\dots,Q\}$，其中 $Q = 2^{b-1}-1$
- 缩放因子（scale）：$s > 0$
- 量化/反量化过程：

$$
q_i = \text{round}(w_i / s), \quad
  \hat{w}_i = s \cdot q_i
$$


典型目标是最小化量化误差：

$$
\min_s \sum_i (w_i - \hat{w}_i)^2
$$


这实际上是一个一维凸优化问题。

工程上常见 heuristics：

- 直接取：

$$
s = \frac{\max_i |w_i|}{Q}
$$


- 或取某个分位数（如 99.9%）替代 max，防止 outlier 过大：

$$
s = \frac{\text{quantile}_{p}(|w_i|)}{Q}
$$


**【要点】**  
你需要知道：

- 量化参数（scale/zero-point）可以通过最小二乘意义下的优化求得
- 工程实现中用 histogram + 搜索 / heuristics 做近似求解

---

### 2.2.2 LSQ（Learned Step Size Quantization）思想

更进一步，可以把 scale $s$ 当成可学习参数，让反向传播直接优化：

- 定义一个“伪量化”算子（用 STE 逼近梯度）
- 在训练/微调时 joint optimize $W$ 和 $s$

形式上：

$$
\min_{W,s} L(\text{Quantize}(W; s))
$$


这种方法使得量化参数针对当前任务/数据集自适应地收敛到较好的值，从而提升低 bit 量化的精度。

---

## 2.3 知识蒸馏（Distillation）的优化视角

给定 teacher 模型 $T$ 和 student 模型 $S$：

- Teacher 输出：$p^T$
- Student 输出：$p^S$

常见的 distillation loss：

$$
L_{\text{distill}} = \alpha \cdot \text{CE}(y, p^S)
+ (1 - \alpha) \cdot T^2 \cdot \text{KL}\big(\sigma(z^T/T) \,\|\, \sigma(z^S/T)\big)
$$


其中：

- CE：交叉熵
- KL：KL 散度
- $T$：温度（temperature）
- $\sigma$：softmax

从优化角度看：

- 这是一个带有两部分损失的目标函数
- 你在“精确拟合 ground-truth 标签”和“模仿 teacher 的软分布”之间做权衡

**【与推理加速的关系】**  
- Student 模型通常更小、更浅、维度更低
- 在边缘设备上部署时会大幅降低延迟与内存占用
- 你要理解：蒸馏可以看作是一种“在 teacher 的函数空间附近寻找浅网络逼近”的优化过程

---

## 2.4 多目标优化：精度 vs 延迟 vs 内存

在做部署时，你不会只关心 loss，还会关心：

- latency（推理延迟）
- memory（显存 / DRAM 占用）
- throughput（QPS）

可以将这些纳入优化目标：

$$
\min_{\theta} \quad \alpha \cdot \text{Error}(\theta)
+ \beta \cdot \text{Latency}(\theta)
+ \gamma \cdot \text{Memory}(\theta)
$$


其中：

- $\theta$：模型结构 + 量化配置 + 剪枝策略 等
- $\text{Latency}(\theta)$：通过 profile 或 analytical model 估计
- $\text{Memory}(\theta)$：由参数量、activation、KV cache 决定

工程实现中常见简化：

- 固定某个 latency/memory 上限作为约束：

$$
\min_{\theta} \text{Error}(\theta)
  \quad \text{s.t.} \quad \text{Latency}(\theta) \le L_{\max}, \
  \text{Memory}(\theta) \le M_{\max}
$$


- 或把 latency/memory 转换为正则项加入 loss

> 重要的是：你要能把“部署需求”翻译成数学上的“目标 + 约束”，这样才能用优化工具系统地设计算法。

---

## 2.5 本章小结

本章重点：

1. 剪枝可以看作带 $L_0/L_1$ 稀疏约束的优化问题
2. 量化可以通过最小化量化误差的优化问题确定 scale/zero-point，进一步可通过训练 joint optimize
3. 蒸馏是一个联合最小化 CE 和 KL 的多目标优化问题
4. 部署时的精度、延迟、内存可以统一本为多目标/带约束的优化问题

> 在后续章节中，这些优化目标会与近似理论、概率统计、硬件模型结合在一起，形成一套完整的推理加速思维。

# 第 3 章：近似理论（Approximation Theory for Quantization & Cheap Ops）

近似理论回答的问题是：

> “在允许一定误差的前提下，如何用更简单的函数 / 更粗的精度来逼近原函数？”

在推理加速中主要应用于：

- 量化误差建模
- 激活函数近似（GELU/SiLU/tanh 等）
- 用多项式 / 分段线性函数替代复杂算子
- approximate computing（例如用 cheap ops 替换 expensive ops）

---

## 3.1 均匀量化的误差模型

考虑步长为 $\Delta$ 的均匀量化：

$$
\hat{w} = Q(w) = \Delta \cdot \text{round}\Big(\frac{w}{\Delta}\Big)
$$


定义量化误差：

$$
e = w - \hat{w}
$$


在很多假设下（信号在每个量化区间内分布比较“均匀”），可以近似认为：

- $e$ 在 $[-\Delta/2, \Delta/2]$ 上均匀分布
- 即：$e \sim \mathcal{U}(-\Delta/2, \Delta/2)$

于是：

$$
\mathbb{E}[e] \approx 0, \quad
\mathbb{E}[e^2] = \text{Var}(e) \approx \frac{\Delta^2}{12}
$$


**【工程意义】**  
- 量化步长 $\Delta$ 越大 → 误差方差越大
- 对某层使用更低 bit（更少级别）时，可以估算噪声能量：$\sigma_e^2 \approx \Delta^2/12$

这为“按层动态选择 bit 宽”提供了理论依据：

- 对更敏感的层使用小 $\Delta$（更多 bit）
- 对不敏感的层使用大 $\Delta$（更少 bit）

---

## 3.2 激活函数的近似：以 GELU 为例

GELU 定义：

$$
\text{GELU}(x) = x \cdot \Phi(x)
$$


其中 $\Phi(x)$ 是标准正态分布的 CDF：

$$
\Phi(x) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^x e^{-t^2/2} \, dt
$$


直接计算 CDF 非常昂贵，因此常用近似：

$$
\text{GELU}(x) \approx 0.5 x \left(1 + \tanh\left[\sqrt{\frac{2}{\pi}}(x + 0.044715 x^3)\right]\right)
$$


进一步，可以将 tanh 又近似为一个多项式或分段线性函数，使整体计算只包含：

- 加法
- 乘法
- 少量表查

**【近似理论视角】**  

- 在一个有限区间 $[a,b]$ 上，用多项式 $P_n(x)$ 逼近一个平滑函数 $f(x)$ 是可行的：

$$
\|f - P_n\|_{\infty, [a,b]} \to 0 \quad \text{(当 n → ∞ 时)}
$$


- 对应到工程中，就是选取一个多项式阶数 n，使得：
  - 逼近误差足够小（精度要求）
  - 计算代价足够低（乘法/加法次数）

---

## 3.3 多项式近似的构造（最小二乘 + Chebyshev）

给定函数 $f(x)$，希望在区间 $[a,b]$ 上用多项式：

$$
P_n(x) = \sum_{k=0}^n c_k x^k
$$


来逼近它。

常见方法：

1. **最小二乘拟合**（离散点）：
   - 在区间内采样若干点 $x_i$，最小化：

$$
\sum_i (f(x_i) - P_n(x_i))^2
$$


   - 这会导致一个线性最小二乘问题求 $\{c_k\}$

2. **Chebyshev 逼近**（均匀最大误差最小）：
   - 用 Chebyshev 多项式作为基函数
   - 在理论上能提供更好的最大误差界

**【对推理工程师的要求】**  
- 不需要会手推 Chebyshev 多项式
- 需要知道：
  - 激活函数/特殊函数的近似来自“多项式拟合”或“有理函数拟合”
  - 阶数越高、基函数越复杂 → 精度越好，但计算也越贵

---

## 3.4 Approximate Computing：用“更便宜”的算子替代

典型思路：

- 用 shift 替代乘法（当系数接近 $2^k$ 时）
- 用 LUT（查表）替代复杂函数（如 exp / log / erf）
- 用 piecewise linear 替代 smooth 非线性（如 ReLU6, HardSwish 等）

这都可以看作是：

> 在允许一定误差的前提下，用“更便宜的算子组合”近似原函数。

**【与你的工作关系】**  

- 在设计自定义 kernel 时，你可以：
  - 通过数学近似把复杂算子分解成“mul+add+max+shift+table lookup”
  - 利用硬件对这些基本算子的高速支持
- 在评估近似时，你需要有：
  - 定性直觉：误差对下游的影响（例如激活输出范围变窄/变宽）
  - 简单定量工具：最大误差、L2 误差等

---

## 3.5 本章小结

1. 近似理论为量化误差提供了噪声模型（例如 $\Delta^2/12$）
2. 激活函数（如 GELU）的工程实现广泛依赖多项式/有理函数近似
3. Approximate computing 本质是“用更便宜的函数族逼近原函数族”
4. 作为推理加速工程师，你需要：
   - 看懂“误差 vs 计算代价”的 tradeoff
   - 在部署时合理选择近似等级，而不是“盲目追求低误差”或“盲目省算力”

后续章节，会用概率/信息论的视角进一步理解这些误差是如何在系统中传播的。


# 第 4 章：概率与统计（Probability & Statistics for Runtime Inference）

本章重点：

- 量化和剪枝中的“分布视角”
- 蒸馏和温度缩放中的 KL/交叉熵
- 通过分位数、直方图选择 clipping 阈值

---

## 4.1 输出分布与 KL 散度

### 4.1.1 交叉熵与 KL

给定真实分布 $p$ 与模型分布 $q$：

- 交叉熵：

$$
H(p, q) = - \sum_i p_i \log q_i
$$


- KL 散度：

$$
\text{KL}(p\|q) = \sum_i p_i \log \frac{p_i}{q_i}
$$


二者关系：

$$
H(p, q) = H(p) + \text{KL}(p\|q)
$$


其中 $H(p)$ 与参数无关，优化时常直接最小化交叉熵。

在蒸馏中，teacher 分布 $p^T$，student 分布 $p^S$：

$$
L_{\text{KD}} = \text{KL}(p^T \| p^S)
$$


反映 student 在多大程度上“模仿”了 teacher 的输出分布。

---

### 4.1.2 温度（Temperature）

logits $z$ 经过 softmax：

$$
p_i = \frac{e^{z_i/T}}{\sum_j e^{z_j/T}}
$$


- $T=1$：常规 softmax
- $T>1$：分布变平滑（“软标签”）
- $T<1$：分布变更尖锐

在蒸馏中通常用 $T>1$：

- Teacher 输出变得更“软”，包含更多类别间相对关系信息
- Student 更容易学习到 teacher 的“暗知识”（dark knowledge）

**【与你的工作关系】**  
- 在边缘部署 Distilled 模型时，你应当理解：
  - student 的行为不仅拟合 hard label，也拟合了 teacher 提供的 soft label
  - 在某些分布 shift 场景中，蒸馏过的模型可能更鲁棒

---

## 4.2 量化中的统计视角：直方图与分位数

在选择量化阈值（clipping range）时，常见的做法是：

1. 收集一段时间的 activations / weights
2. 画直方图（histogram）
3. 决定：
   - 是否使用对称量化（symmetric）
   - 是否 clip 例如 top 0.1% 的 outliers

### 4.2.1 分位数剪裁（percentile clipping）

例如设置：

$$
\alpha = \text{quantile}_{p}(|x|)
$$


然后将范围裁剪到 $[- \alpha, \alpha]$，再在该范围内做均匀量化。

这样做的理由是：

- 极少数非常大的 outlier 会极大放大 scale，从而让大部分值被挤压到极细的 quantization bin
- 将 outlier 裁剪掉通常对最终性能影响不大，有时甚至有正则化效果

---

### 4.2.2 KL-based threshold 选择

部分框架（例如早期的 TensorRT / TensorFlow 量化方案）会：

- 尝试多个不同的 clipping threshold
- 对每个阈值，将原始直方图与量化后重建的直方图比较
- 选择使 KL(p\|q) 最小的阈值

**【本质】**  
通过 KL 来衡量分布形状的变化，避免量化导致分布严重“畸变”。

---

## 4.3 统计稳健性（Robust Statistics）

在推理部署中，你需要考虑：

- 上游数据分布是否会发生漂移（distribution shift）
- 是否存在噪声、异常值（outliers）
- 对不同层的激活分布要有宏观认知：
  - 是否近似高斯
  - 是否有长尾

有些技术（例如 robust norm、Huber 损失）可以缓解 outlier 对参数/统计量的影响。  
在这里你不需要深入推导，只需要知道：

- 许多可以“抑制 outlier”的方法本质上来自稳健统计理论
- 在量化/clip/归一化中，使用分位数而不是 max/min 也是一种稳健统计实践

---

## 4.4 本章小结

1. KL、交叉熵是蒸馏与分布对齐的数学工具
2. 温度缩放（T）控制输出分布的“平滑度”，影响学习信号
3. 量化中的阈值选择通常基于直方图与分位数剪裁
4. 稍有统计直觉，有助于理解：
   - 为什么 outlier 会毁掉量化
   - 为什么剪裁 outlier 反而可能更好
   - 为什么蒸馏后的模型在部署中可能更鲁棒

后续章节的信息论会从更基础的角度解释熵与压缩的关系。


# 第 5 章：信息论（Information Theory for Compression）

信息论回答的问题是：

> “要表示一个随机变量/信号最少需要多少 bit？”

在推理压缩中，信息论提供：

- 熵（entropy）：理论最小 bit 宽
- rate–distortion：压缩率 vs 失真度
- 作为模型剪枝 / 量化的一种“上帝视角”

---

## 5.1 熵（Entropy）

离散随机变量 $X$ 取值 $\{x_i\}$，概率 $p_i$，其熵定义为：

$$
H(X) = -\sum_i p_i \log_2 p_i
$$


- 单位是 bits
- 反映“平均需要多少 bit 来编码 X 的取值”

**例子**：硬币正反面等概率：

$$
H(X) = - (0.5 \log_2 0.5 + 0.5 \log_2 0.5) = 1 \text{ bit}
$$


---

## 5.2 与量化 / 编码的关系

如果你对权重/激活做了量化和统计建模：

- 量化后值域有限，例如 $\{-127,\dots,127\}$
- 但每个值出现频率不同
- 你可以用 entropy coding（Huffman / arithmetic coding）进一步压缩比特流

理论上，平均码长下界接近熵：

$$
\mathbb{E}[\text{code length}] \ge H(X)
$$


**【工程视角】**  
- 这解释了：
  - 为什么稀疏矩阵 / 集中分布的值更好压缩
  - 为什么有些 post-training compression 工具可以在不影响精度的前提下，大幅减少模型文件大小（对部署包很关键）

---

## 5.3 Rate–Distortion（率失真）

Rate–distortion 理论研究：

> 在平均失真不超过 D 的前提下，最小的比特率 R 是多少？

通俗版本：

- “我要压小一点，愿意损失多少精度？”
- 或反过来：“给我这么多 bit，我最多能守住多少质量？”

虽然在部署端不会直接用上正式的 rate–distortion 函数，  
但这个思路在很多论文/方法论中都反复出现：

- bit allocation（给不同层/通道分配不同 bit）
- 剪枝比例分配（给不同层分配不同稀疏度）

可以理解为：

> 一个受限资源（bit budget / FLOPs budget）下的“优化分配问题”，信息论提供对“最优情况”的一个理论参考。

---

## 5.4 本章小结

1. 熵度量了平均所需的编码 bit 数
2. 熵越低 → 越容易压缩（更高压缩比）
3. 量化 + 熵编码可以进一步减少模型大小
4. Rate–distortion 提供了一种“资源–损失”的系统性思考方式

在具体部署中，你不用推导定理，但可以用这些概念来：

- 解释“为什么这层可以 aggressively 压缩，而那层不行？”
- 理解“为什么压缩工具能做到看似‘白嫖’空间”。


# 第 6 章：信号处理与卷积数学（Signal Processing & Convolution）

本章连接：

- 信号处理视角下的卷积
- FFT、Winograd 等快速卷积算法
- Toeplitz / Circulant 等结构化矩阵

这些数学为卷积加速提供理论基础。

---

## 6.1 卷积定理：时域卷积 = 频域乘法

一维离散卷积：

$$
y[n] = \sum_k x[k] h[n-k]
$$


其离散时间傅里叶变换（DTFT）满足：

$$
Y(\omega) = X(\omega) H(\omega)
$$


这就是 **卷积定理**：

> 卷积在频域里变成点乘。

计算复杂度：

- 直接卷积：$O(NK)$
- FFT-based：
  - $O(N \log N)$ 计算 FFT/逆 FFT
  - 中间是逐点乘法（$O(N)$）

在某些大核/长序列场景，FFT 卷积更划算。

---

## 6.2 Toeplitz 与 Circulant 矩阵

一维卷积可以写成 Toeplitz 矩阵与向量乘法：

$$
y = T x
$$


其中 Toeplitz 矩阵 T 的每条对角线元素相同。  
在周期边界条件下可变成 Circulant 矩阵，其特征：

- 可以被傅里叶基对角化
- 矩阵–向量乘法可以通过 FFT 高效实现

$$
C = F^\ast \Lambda F
$$


- F：离散傅里叶变换矩阵
- $\Lambda$：对角矩阵（频域增益）

**【工程视角】**  

- 理解 Toeplitz/Circulant 结构有助于：
  - 理解 FFT 卷积的本质
  - 理解某些“频域参数化”的卷积层
- 不需要记住复杂推导，但要知道：
  - 卷积可以被看成带特殊结构的矩阵
  - 特殊结构可以带来更快的乘法算法

---

## 6.3 Winograd 卷积（最小乘法算法）

Winograd 思想：

> 用更多的加法/减法和少量变换，换取更少的乘法次数。

例如 F(2×2, 3×3) 算法：  
- 输入 tile 大小：4×4
- 输出 tile 大小：2×2
- 通过对输入和卷积核做线性变换，将卷积转化为：
  - element-wise 乘法
  - 再经过逆变换

其优点：

- 对小核卷积（如 3×3）乘法数大幅减少
- 在乘法远比加法“贵”的硬件上更有优势

缺点：

- 数值稳定性可能变差（放大误差）
- 对大核/stride 不友好
- 实现复杂

**【推理加速视角】**  

- 你需要知道：
  - 许多高性能 Conv kernel（尤其在移动端）使用 Winograd 或其变体
  - 它们都是在线性代数/多项式插值基础上得到的最小乘法算法

---

## 6.4 本章小结

1. 卷积定理：时域卷积 ↔ 频域乘法，为 FFT 卷积提供数学基础
2. Toeplitz/Circulant 矩阵提供了“卷积是结构化矩阵乘法”的视角
3. Winograd 算法通过巧妙变换减少乘法次数，是很多 Conv 加速方法的核心

在实际部署中，你不一定亲自实现 FFT/Winograd kernel，  
但理解其数学，有助于：

- 解读不同硬件/库对卷积的性能差异
- 做出“某些卷积形状适配某些算法”的合理判断。


# 第 7 章：计算图与图论（Computational Graph & Graph Theory）

本章关注：

- 如何用 DAG（有向无环图）表示模型
- graph rewrite / fusion 的数学抽象
- 如何基于图结构做 scheduling / partitioning

---

## 7.1 模型即计算图（DAG）

在框架中，一个模型通常被表示为：

- 节点（node）：算子（op），例如 MatMul、Add、GELU、LayerNorm 等
- 边（edge）：张量数据流

由于不存在环（没有算子依赖未来的结果），该图是一个 **DAG**。

数学上，DAG 性质保证：

- 可以进行 **拓扑排序（topological sort）**
- 存在至少一种执行顺序使得所有依赖都满足

---

## 7.2 图重写（Graph Rewrite）

图重写的目标：

> 在不改变整体输入–输出语义的前提下，改写图结构以获得更高效率。

常见操作：

1. **算子融合（Operator Fusion）**

   例如：

$$
Y = \text{GELU}(X W + b)
$$


   在图中通常表现为：`X → MatMul → Add → Gelu` 四个节点。

   通过图重写，可以识别这是一种常见 pattern，将其替换为：

   - 单个 `FusedMatMulBiasGelu` 节点
   - 内部实现为一个 kernel，减少中间内存读写

2. **常量折叠（Constant Folding）**

   若某些节点仅依赖常量张量，可以在编译时提前执行，简化推理时的计算量。

3. **消除冗余算子（Elimination）**

   - 连续的 reshape/transpose 合并
   - 反向操作互相抵消（如 transpose 两次）

**数学抽象**：

- 一组 rewrite rule：

$$
P \Rightarrow R
$$


  其中 P 是 pattern 子图，R 是替换后的子图。

- 在 DAG 上遍历，匹配 pattern，进行等价替换。

---

## 7.3 图划分与并行（Partitioning & Parallelism）

对大模型，尤其在多设备/多核场景下，需要将计算图划分为子图：

- pipeline parallelism
- tensor parallelism
- expert parallelism（MoE）

数学上涉及：

- 图划分（graph partitioning）：
  - 希望 cut 边尽量少（减少通信）
  - 各子图负载尽量平衡（避免一块成为瓶颈）
- 负载均衡问题本质上是一个 NP-hard 的组合优化问题
- 实际使用启发式算法（greedy / spectral partitioning / METIS 等）

**【与你的工作关系】**  

- 在 edge device 上，多核/多引擎并行也需要合理切图
- 推理框架常会自动做 operator placement，你需要能看懂“为什么某些 op 被放在某个引擎上”

---

## 7.4 Liveness 分析与内存规划（Memory Planning）

计算图还承载：

- 每个张量的 **生命周期（liveness interval）**
- 哪些张量在同一时间段内“同时活跃”

基于此可做：

- 内存重用（buffer reuse）
- in-place 计算

从数学上，这类似于：

- 图着色 / 区间图着色（interval graph coloring）
- 目标是最少的“颜色”（内存块）覆盖所有区间

> 虽然你不会直接实现图着色算法，但要知道内存规划问题在图论中是一个经典主题。

---

## 7.5 本章小结

1. 模型可以抽象为一个 DAG；拓扑排序提供执行顺序
2. 图重写（fusion/folding/elimination）是推理加速的关键步骤
3. 图划分与并行涉及图划分问题，需要平衡通信与负载
4. 内存规划可以用 liveness 分析 + 图着色的视角理解

从这一章开始，数学不再是“公式推导”，而是更多地提供了一种 **抽象建模工具**，帮助你从更高层理解编译器与推理框架在干什么。


# 第 8 章：数值稳定性与复杂度（Numerical Stability & Complexity）

本章关注：

- 浮点数的误差模型
- 稳定 softmax / LayerNorm
- 算法复杂度（时间/空间）与推理性能

---

## 8.1 浮点数与舍入误差

一个典型的浮点运算可以抽象为：

$$
\text{fl}(x \circ y) = (x \circ y)(1 + \delta), \quad |\delta| \le \epsilon
$$


- $\circ$：基本运算（+,-,×,÷）
- $\epsilon$：机器精度（machine epsilon）
- $\delta$：舍入误差

连续多次运算后，误差会积累，有时还会被放大。

**【工程意义】**  

- 在 FP32 → FP16/FP8 量化时，$\epsilon$ 变大  
- 在长序列、深网络中，累积误差不容忽视
- 需要设计数值稳定的实现方式（见 softmax 示例）

---

## 8.2 稳定 softmax

朴素写法：

$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$$


可能遇到：

- 当某个 $x_k$ 很大时，$e^{x_k}$ 溢出
- 其它项相对变成 0，导致数值问题

稳定写法：

$$
\text{softmax}(x_i) = \frac{e^{x_i - m}}{\sum_j e^{x_j - m}}, \quad m = \max_j x_j
$$


推导：

$$
\frac{e^{x_i}}{\sum_j e^{x_j}}
= \frac{e^{x_i - m} e^m}{\sum_j e^{x_j - m} e^m}
= \frac{e^{x_i - m}}{\sum_j e^{x_j - m}}
$$


- 分子分母同时乘以 $e^{-m}$，数值上避免了溢出/下溢

**【工程视角】**  

- 在 FP16/FP8 或混合精度中，这种稳定化处理尤其重要
- FlashAttention 等算法在实现时非常强调：
  - 按 block 计算 max
  - 增量式维护稳定 softmax

---

## 8.3 LayerNorm / RMSNorm 的数值考虑

LayerNorm：

$$
\mu = \frac{1}{d}\sum_{i=1}^d x_i, \quad
\sigma^2 = \frac{1}{d}\sum_{i=1}^d (x_i - \mu)^2 \\
\hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}}
$$


数值稳定性挑战：

- $\sigma^2$ 很小或很大时，都可能出现精度损失
- FP16 中的累加（sum）不精确

常见对策：

- 在高精度（FP32）accumulator 中累加，再 cast 回低精度
- 使用 Kahan summation 改善求和精度（较少见于实际 ML kernel，但思想值得知道）

---

## 8.4 算法复杂度（Complexity）

在推理场景中，关注的复杂度包括：

- 时间复杂度：FLOPs 数 / 算法阶
- 空间复杂度：参数量 / activation / KV cache

例：自注意力 $O(T^2 d)$：

- QKᵀ：$O(T^2 d)$
- A V：$O(T^2 d)$

当 T 很大时（例如长上下文 LLM），这是主要瓶颈。  
因此：

- 许多“线性注意力”、“局部注意力”、“稀疏注意力”方法试图将复杂度降到 $O(T d)$ 或 $O(T \log T)$
- 你需要能大致估算不同模块的 FLOPs，判断瓶颈在哪一层

---

## 8.5 本章小结

1. 浮点误差模型帮助你理解：为什么低精度需要数值稳定技巧
2. 稳定 softmax / LayerNorm 是经典例子，推理 kernel 必须实现稳定版本
3. 算法复杂度决定了模型在大规模输入/长序列下的 scalability
4. 在做 runtime acceleration 时：
   - 一部分工作是 **减少常数项**（kernel 级优化）
   - 另一部分是 **降低复杂度阶数**（模型/算法级优化）


# 第 9 章：硬件相关数学（Hardware-Aware Math: Roofline, Tiling, SIMD, FMA）

本章聚焦：

- Roofline 模型：算力 vs 带宽
- 算术强度（arithmetic intensity）
- 分块（tiling）与数据重用
- SIMD / FMA / GEMM 内核的数学视角

---

## 9.1 Roofline 模型

Roofline 模型给出在给定硬件上，某个 kernel 的理论性能上界：

$$
P_{\text{attainable}} = \min\left(P_{\text{peak}},\ I \cdot B\right)
$$


其中：

- $P_{\text{peak}}$：峰值计算性能（如 TFLOPs）
- $B$：内存带宽（如 GB/s）
- $I$：算术强度（arithmetic intensity）

算术强度定义为：

$$
I = \frac{\text{FLOPs}}{\text{Bytes moved}}
$$


**两种 regime：**

1. **memory-bound**（带宽受限）：
   - 若 $I \cdot B < P_{\text{peak}}$
   - 性能主要受内存带宽限制，FLOPs 很充裕
2. **compute-bound**（算力受限）：
   - 若 $I \cdot B \ge P_{\text{peak}}$
   - 性能由算力上限决定

**【工程视角】**  

- 对算子而言：
  - Conv/GEMM 通常具有较高 $I$，更容易成为 compute-bound
  - 简单 element-wise op 的 $I$ 很低，通常是 memory-bound
- 对 Embedding 而言：几乎没有 FLOPs，但读写大量数据 → 典型 memory-bound

---

## 9.2 分块（Tiling）与数据重用

以 GEMM 为例：$C = A B$，其中：

- $A \in \mathbb{R}^{M \times K}$
- $B \in \mathbb{R}^{K \times N}$
- $C \in \mathbb{R}^{M \times N}$

如果直接三重循环，会频繁从内存加载 A/B/C，导致带宽浪费。

分块思想：

- 将 C 分成小块（tile）：$M_b \times N_b$
- 对每个 tile：
  - 加载对应的 A 子块（$M_b \times K_b$）
  - 加载对应的 B 子块（$K_b \times N_b$）
  - 在寄存器或小缓存中完成所有 FMA

约束条件：

$$
(M_b \cdot K_b + K_b \cdot N_b + M_b \cdot N_b) \cdot \text{bytes} \le \text{cache size}
$$


**【直观理解】**  

- 通过分块，让同一块数据被多次重用（reuse），有效提高算术强度 $I$
- 减少对 DRAM 的访问次数

---

## 9.3 SIMD / FMA / Tensor Core

### 9.3.1 FMA 再回顾

$$
\text{FMA}(a,b,c) = a \times b + c
$$


- 在硬件中可以作为一个基本指令
- dot product 与 GEMM 内部都是大量 FMA

### 9.3.2 SIMD（Single Instruction, Multiple Data）

SIMD 指令可以：

- 一条指令对多个数据元素执行相同运算
- 例如 AVX2 中 256-bit 寄存器可同时处理 8 个 float

矩阵乘法内部：

- 每次从内存加载一个向量 block
- 使用 SIMD/FMA 对多个元素并行乘加

### 9.3.3 Tensor Core / Matrix Unit

在现代 GPU/NPU 中：

- 提供专门的矩阵乘单元（如 NVIDIA Tensor Core、Intel AMX）
- 每条指令执行一个小矩阵乘运算（如 16×16 × 16×16）

**【数学视角】**  

- 这些专用单元本质上实现了一个：

$$
C_{\text{tile}} = A_{\text{tile}} B_{\text{tile}} + C_{\text{tile}}
$$


- 你的任务是：
  - 把大矩阵拆成符合硬件 tile 大小的小块
  - 做好数据布局，使得这些 tile 在内存中连续、对齐

---

## 9.4 硬件意识下的量化与打包（Packing）

对 INT8 / INT4 GEMM：

- 权重/激活不仅被量化，还会按特定 pattern 打包（pack）到寄存器友好的格式
- 例如把多个 int4 压在一个 16-bit 或 32-bit 容器中

这涉及：

- 位运算（bitwise ops）
- 对齐（alignment）约束
- 多个元素的“并行 unpack”数学

虽然这里不展开细节，但要知道：

> 高性能量化推理内核的数学本质是：  
> **用整数算术和位运算实现“向量化 FMA”的等价行为。**

---

## 9.5 本章小结

1. Roofline 模型帮助你判断 kernel 是 compute-bound 还是 memory-bound
2. 算术强度 $I$ 是理解“为什么要做 tiling / fusion / 缓存重用”的关键指标
3. FMA、SIMD、Tensor Core 是高性能线性代数的硬件支柱
4. 量化推理中还要考虑整数打包与位运算的数学结构

> 到这里为止，你已经拥有了一套：  
> **从线性代数 → 优化 → 近似 → 统计 → 信息论 → 信号处理 → 图论 → 数值分析 → 硬件数学** 的完整思维链条。

在实际工程中，每一次做部署决策，你都可以在脑中快速走一遍这条链：

- 这个算子本质是什么线性代数？
- 能不能用压缩/近似/剪枝优化？
- 对分布/信息量有何影响？
- 会不会引发数值问题？
- 在硬件上是 compute-bound 还是 memory-bound？
- 该如何做 tiling / fusion / placement？


# 附录 A：符号表与术语速查

- $W, A, B, C$：矩阵
- $x, y, z$：向量
- $\|\cdot\|_F$：Frobenius 范数
- $\|\cdot\|_2$：谱范数 / operator norm
- $\|\cdot\|_1, \|\cdot\|_0$：L1 范数、L0 “范数”（非零个数）
- SVD：Singular Value Decomposition
- GEMM：General Matrix-Matrix Multiply
- FMA：Fused Multiply-Add
- AI：Arithmetic Intensity（算术强度）
- KL：Kullback-Leibler 散度
- CE：Cross-Entropy 交叉熵
- FFT：Fast Fourier Transform
- DAG：Directed Acyclic Graph（有向无环图）

---

# 附录 B：练习建议（自我检查）

你可以按照下面的 checklist 检查自己是否掌握了本书核心内容：

1. **线性代数**
   - 能写出 GEMM 的元素级公式，并解释其 FLOPs 规模
   - 能解释 SVD 截断是最优低秩近似，并画出示意图
   - 能从 Conv2d 推导出 im2col + GEMM 形式

2. **数值优化**
   - 能写出带 L0/L1 正则的剪枝目标函数
   - 能解释如何通过最小化量化误差来求 scale
   - 能解释蒸馏中 CE + KL 的含义

3. **近似理论**
   - 能写出均匀量化误差的方差 $\Delta^2/12$
   - 能解释为什么激活函数可以用多项式近似

4. **概率与统计 / 信息论**
   - 理解 KL、交叉熵的定义及其关系
   - 能解释熵与平均码长的关系

5. **信号处理 / 图论 / 数值稳定 / 硬件数学**
   - 理解卷积定理与 FFT 卷积大致思路
   - 知道 DAG、graph rewrite 的意义
   - 能解释稳定 softmax 的推导
   - 能用 Roofline 模型说清楚一个 kernel 是 compute-bound 还是 memory-bound

---

> 建议：  
> 你可以在本 notebook 的基础上：
> - 增加代码 cell，做一些数值实验（例如 SVD 压缩一个随机矩阵，观察误差 vs rank）  
> - 写一些小脚本 profiling 不同形状的 GEMM，感受 tiling 的影响  
> - 整合你的 GM 实际部署案例，逐渐把这里变成你自己的“Runtime Inference 数学工作手册”。
